<a href="https://colab.research.google.com/github/meerilahi/algoTrading/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install alpha_vantage

In [108]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
%matplotlib inline

In [121]:
API_KEY = "XM0ZE0M05HEDFVTP"
ts = TimeSeries(key = API_KEY, output_format = 'pandas')
data = ts.get_daily(symbol = 'MSFT', outputsize = 'full')[0]

In [ ]:
df = data[['4. close']]
df = df.rename(columns = {'4. close':'target'})
df = df[::-1]
for i in range(1,31):
    df[f'lag_{i}'] = df['target'].shift(i)
df = df.dropna()
df = df.iloc[::,::-1]

In [131]:
mean_squared_error(df['lag_1'], df['target'])
mean_absolute_error(df['lag_1'], df['target'])

1.257317556290348

In [137]:
X = df.iloc[:,0:-1].to_numpy()
X = X.reshape(X.shape[0], X.shape[1], 1)
y = df.iloc[:,-1].to_numpy()

In [138]:
split_index = int(len(X) * 0.8)
X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [142]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape = (30, 1)),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam',loss='mse', metrics=['mean_absolute_error'])
model.fit(train_ds.batch(32), epochs = 23,validation_data = test_ds.batch(32))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [176]:
model.save("model_copy.keras")
model2 = tf.keras.models.load_model("model_copy.keras")

In [178]:
X_test.shape, y_test.shape

((1271, 30, 1), (1271,))

In [177]:
y_pred = []
for i in range(len(X_test)):
    y_pred.append(model2.predict(X_test[i].reshape(1,30,1)))
    print('Example' + str(i))
    print('y_pred: ' + str(y_pred[i]) + ' -  y_test: ' + str(y_test[i]))
    model2.fit(X_test[i].reshape(1,30,1), y_test[i].reshape(1,1), epochs = 1)

Streaming output truncated to the last 5000 lines.
Example21
y_pred: [[139.13809]] -  y_test: 146.92
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 60.5581 - mean_absolute_error: 7.7819
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Example22
y_pred: [[136.79726]] -  y_test: 156.11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 372.9821 - mean_absolute_error: 19.3127
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Example23
y_pred: [[140.78445]] -  y_test: 149.7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 79.4869 - mean_absolute_error: 8.9155
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Example24
y_pred: [[139.2615]] -  y_test: 160.23
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 439.6776 - mean_absolute_error: 20.9685
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Example25
y_pred: [[143.1793]] -  y_test: 157.71
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 211.1413 - mean_absolute_error: 14.5307
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Example26
y_pred: [[143.47685]] -  y_test: 152.11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/ste

In [180]:
y_pred = np.array(y_pred)
y_pred2 = y_pred.reshape(y_pred.shape[0])

In [ ]:
backtest = pd.DataFrame({
  "pred_close" : y_pred,
  "true_close" : y_test
})
backtest["signal"] = backtest["pred_close"] > backtest["true_close"]

In [ ]:
initial_balance = 10000
balance = initial_balance
shares = 0

for i in range(len(backtest) - 1):
    if backtest["Signal"].iloc[i]:  # Buy
        shares = balance / test["Close"].iloc[i]
        balance = 0
    else:  # Sell
        balance = shares * test["Close"].iloc[i]
        shares = 0

# Final balance
final_balance = balance + (shares * test["Close"].iloc[-1])
profit = final_balance - initial_balance
print(f"Final Balance: ${final_balance:.2f}, Profit: ${profit:.2f}")
